## NVIDIA 加速推理平台介绍 ：https://build.nvidia.com/

# 登录NVIDIA平台获取API Key

In [46]:
import getpass
import os

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

# 展示所有可用models

In [60]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

[Model(id='meta/llama-3.1-8b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='databricks/dbrx-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-dbrx-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='writer/palmyra-fin-70b-32k', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=True, base_model=None),
 Model(id='adept/fuyu-8b', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/adept/fuyu-8b', aliases=['ai-fuyu-8b', 'playground_fuyu_8b', 'fuyu_8b'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mistral-7b-instruct-v0.3', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mistral-7b-instruct-v03'], supports_tools=False, supports_structured

# 尝试直接问弱智吧问题 -- 与预期不符

In [92]:
from openai import OpenAI

llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")
result = llm.invoke("为什么烧水只敢烧到99度？")
print(result.content)


答案：因为100度要被开水烫伤呗


# 导入所需库

In [93]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from operator import itemgetter
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import faiss

# 定义 embedding model

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedding_model = NVIDIAEmbeddings(model="NV-Embed-QA")

# 数据读取和基本数据清理

In [95]:
import os
from tqdm import tqdm
from pathlib import Path

# Here we read in the text data and prepare them into vectorstore
raw_data_dir = "./raw_data/"
ps = os.listdir(raw_data_dir)
data = []
sources = []
for p in ps:
    if p.endswith('.txt'):
        path2file=raw_data_dir+p
        with open(path2file,encoding="utf-8") as f:
            lines=f.readlines()
            for line in lines:
                if len(line)>=1:
                    data.append(line)
                    sources.append(path2file)

documents=[d for d in data if d != '\n']
len(data), len(documents), data[0]

(38, 13, '我烧水只敢烧到99℃ ，防止家里同时出现两个沸物。\n')

# 创建/读取本地FAISS DB

In [96]:
embed_dir = './embedding'

if os.path.exists(embed_dir):
    # Load the vectorestore back.
    update = FAISS.load_local(folder_path=embed_dir, embeddings=embedding_model, allow_dangerous_deserialization=True)

else:
    text_splitter = CharacterTextSplitter(chunk_size=512, separator=" ")
    docs = []
    metadatas = []

    for i, d in enumerate(documents):
        splits = text_splitter.split_text(d)
        #print(len(splits))
        docs.extend(splits)
        metadatas.extend([{"source": sources[i]}] * len(splits))

    store = FAISS.from_texts(docs, embedder , metadatas=metadatas)
    store.save_local(embed_dir)


# 使用RAG获得基于弱智吧QA的回答

In [97]:
retriever = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)
# print(retriever)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("为什么烧水只敢烧到99度？")

tags=['FAISS', 'NVIDIAEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1345888b0>


'防止家里同时出现两个沸物'